In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from sklearn import datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]
Y = iris.target

## Data Generation

In [4]:
print(X.shape)
print(Y.shape)

(150, 2)
(150,)


In [5]:
X = torch.from_numpy(X)
Y = torch.from_numpy(Y)

In [6]:
import torch.utils.data as Data

In [7]:
torch_data = Data.TensorDataset(X, Y)

In [8]:
loader = Data.DataLoader(
    dataset=torch_data,
    batch_size=64,
    shuffle=True,
    num_workers=2,
)

## NN Generation

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 32)
        self.fc2 = nn.Linear(32, 8)
        self.fc3 = nn.Linear(8, 3)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
net = Net()
net = net.float()
print(net)

Net(
  (fc1): Linear(in_features=2, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
)


In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

## Training

In [12]:
for epoch in range(500):
    running_loss = 0.0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        outputs = net(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        #if i == 0:
        #    print(running_loss)
        running_loss = 0.0

In [13]:
outputs = net(X.float())
result = torch.argmax(outputs, 1).numpy() - Y.numpy()
print('error rate:', sum(1 * (result != 0)) / result.shape[0])

error rate: 0.21333333333333335
